In [1]:
from __future__ import print_function
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np

In [2]:
train_data_gen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    rescale=1.0/255.,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    validation_split=0.2,
)

test_data_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

In [4]:
train_data_generator = train_data_gen.flow_from_directory(
    "./archive/data/train",
    target_size=(32,32),
    color_mode='rgb',
    subset='training',
    class_mode='categorical',
    batch_size=64,
    seed=42)

validation_data_gen =train_data_gen.flow_from_directory(
    "./archive/data/train",
    target_size=(32,32),
    color_mode='rgb',
    subset='validation',
     class_mode='categorical',
    batch_size=64,
    seed=42)

test_data_generator = test_data_gen.flow_from_directory(
    "./archive/data/test",
    target_size=[32,32],
    color_mode='rgb',
    batch_size=64,
    seed=42)                              
                                   

Found 62560 images belonging to 46 classes.
Found 15640 images belonging to 46 classes.
Found 13800 images belonging to 46 classes.


In [5]:
train_data_generator.class_indices

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '23': 16,
 '24': 17,
 '25': 18,
 '26': 19,
 '27': 20,
 '28': 21,
 '29': 22,
 '3': 23,
 '30': 24,
 '31': 25,
 '32': 26,
 '33': 27,
 '34': 28,
 '35': 29,
 '36': 30,
 '37': 31,
 '38': 32,
 '39': 33,
 '4': 34,
 '40': 35,
 '41': 36,
 '42': 37,
 '43': 38,
 '44': 39,
 '45': 40,
 '5': 41,
 '6': 42,
 '7': 43,
 '8': 44,
 '9': 45}

In [6]:
import json

with open('class_labels.json', 'w') as outfile:
    json.dump(train_data_generator.class_indices, outfile)

In [7]:
with open('class_labels.json', 'r') as f:
    json_obj = json.load(f)

In [8]:
json_obj

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '23': 16,
 '24': 17,
 '25': 18,
 '26': 19,
 '27': 20,
 '28': 21,
 '29': 22,
 '3': 23,
 '30': 24,
 '31': 25,
 '32': 26,
 '33': 27,
 '34': 28,
 '35': 29,
 '36': 30,
 '37': 31,
 '38': 32,
 '39': 33,
 '4': 34,
 '40': 35,
 '41': 36,
 '42': 37,
 '43': 38,
 '44': 39,
 '45': 40,
 '5': 41,
 '6': 42,
 '7': 43,
 '8': 44,
 '9': 45}

In [9]:
b_classes = 46
nb_epoch = 200

In [16]:
keras.backend.clear_session()
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(46, activation='softmax'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 15, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          3

In [18]:

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)


In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

In [20]:
model.fit(train_data_generator,
              epochs=nb_epoch,
              validation_data=validation_data_gen,
              callbacks=[lr_reducer, early_stopper])

Epoch 1/200
978/978 [==============================] - 3475s 4s/step - loss: 1.4699 - accuracy: 0.5800 - val_loss: 0.7054 - val_accuracy: 0.7977
Epoch 2/200
978/978 [==============================] - 41s 41ms/step - loss: 0.4824 - accuracy: 0.8577 - val_loss: 0.5563 - val_accuracy: 0.8379
Epoch 3/200
978/978 [==============================] - 41s 42ms/step - loss: 0.3387 - accuracy: 0.9017 - val_loss: 0.4864 - val_accuracy: 0.8607
Epoch 4/200
978/978 [==============================] - 41s 42ms/step - loss: 0.2730 - accuracy: 0.9232 - val_loss: 0.4620 - val_accuracy: 0.8677
Epoch 5/200
978/978 [==============================] - 41s 41ms/step - loss: 0.2375 - accuracy: 0.9334 - val_loss: 0.4359 - val_accuracy: 0.8797
Epoch 6/200
978/978 [==============================] - 41s 41ms/step - loss: 0.2067 - accuracy: 0.9426 - val_loss: 0.4287 - val_accuracy: 0.8847
Epoch 7/200
978/978 [==============================] - 41s 41ms/step - loss: 0.1865 - accuracy: 0.9477 - val_loss: 0.4364 - val_ac

In [21]:
model.evaluate(test_data_generator)

216/216 [==============================] - 406s 2s/step - loss: 0.0850 - accuracy: 0.9835


[0.0850026085972786, 0.9834782481193542]

In [22]:
model.save('devanagari_minst.h5')

In [23]:
devnagri_model_json = model.to_json()

In [24]:
with open('devanagri_model.json', 'w') as f:
    f.write(devnagri_model_json)
model.save_weights('dev_nagri_model_weights.h5')